In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline



In [2]:
# Spark oturumunu başlatın
spark = SparkSession.builder.getOrCreate()

# Veri dosyasını yükleyin
data = spark.read.csv(r'C:\Users\MertCan\Desktop\RECOMM\beer_reviews.csv', header=True, inferSchema=True)



In [3]:
data

DataFrame[index: int, brewery_id: int, brewery_name: string, review_time: int, review_overall: double, review_aroma: double, review_appearance: double, review_profilename: string, beer_style: string, review_palate: double, review_taste: double, beer_name: string, beer_abv: double, beer_beerid: int]

In [4]:
data_df = data.toPandas()
data_df

,index,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586609,1586609,14359,The Defiant Brewing Company,1162684892,5.0,4.0,3.5,maddogruss,Pumpkin Ale,4.0,4.0,The Horseman's Ale,5.2,33061
1586610,1586610,14359,The Defiant Brewing Company,1161048566,4.0,5.0,2.5,yelterdow,Pumpkin Ale,2.0,4.0,The Horseman's Ale,5.2,33061
1586611,1586611,14359,The Defiant Brewing Company,1160702513,4.5,3.5,3.0,TongoRad,Pumpkin Ale,3.5,4.0,The Horseman's Ale,5.2,33061
1586612,1586612,14359,The Defiant Brewing Company,1160023044,4.0,4.5,4.5,dherling,Pumpkin Ale,4.5,4.5,The Horseman's Ale,5.2,33061


In [5]:
data_df.isnull().sum()

index                     0
brewery_id                0
brewery_name             15
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename      348
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67785
beer_beerid               0
dtype: int64

In [6]:
data = data.na.drop()

In [7]:
data_df=data.toPandas()
data_df

,index,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518473,1586609,14359,The Defiant Brewing Company,1162684892,5.0,4.0,3.5,maddogruss,Pumpkin Ale,4.0,4.0,The Horseman's Ale,5.2,33061
1518474,1586610,14359,The Defiant Brewing Company,1161048566,4.0,5.0,2.5,yelterdow,Pumpkin Ale,2.0,4.0,The Horseman's Ale,5.2,33061
1518475,1586611,14359,The Defiant Brewing Company,1160702513,4.5,3.5,3.0,TongoRad,Pumpkin Ale,3.5,4.0,The Horseman's Ale,5.2,33061
1518476,1586612,14359,The Defiant Brewing Company,1160023044,4.0,4.5,4.5,dherling,Pumpkin Ale,4.5,4.5,The Horseman's Ale,5.2,33061


In [10]:
# Gerekli sütunları seçin
beer_data_id_name = data.select('beer_beerid', 'beer_name')

# Tekil değerleri alın
beer_data_id_name = beer_data_id_name.distinct()
beer_data_id_name=beer_data_id_name.toPandas()
beer_data_id_name.to_csv("C:\\Users\\MertCan\\Desktop\\RECOMM\\name_id.csv", index=False)

In [8]:
# Gerekli sütunları seçin
beer_data = data.select('beer_beerid','beer_name','beer_abv', 'review_profilename', 'review_taste','review_palate','review_aroma','review_appearance','beer_style', 'review_overall')
 # NaN değerleri temizleyin
beer_data = beer_data.na.drop()


In [9]:
beer_data_df = beer_data.toPandas()

In [18]:
beer_data_df.to_csv("C:\\Users\\MertCan\\Desktop\\RECOMM\\output.csv", index=False)

In [10]:
beer_data.toPandas().head()

,beer_beerid,beer_name,beer_abv,review_profilename,review_taste,review_palate,review_aroma,review_appearance,beer_style,review_overall
0,47986,Sausa Weizen,5.0,stcules,1.5,1.5,2.0,2.5,Hefeweizen,1.5
1,48213,Red Moon,6.2,stcules,3.0,3.0,2.5,3.0,English Strong Ale,3.0
2,48215,Black Horse Black Beer,6.5,stcules,3.0,3.0,2.5,3.0,Foreign / Export Stout,3.0
3,47969,Sausa Pils,5.0,stcules,3.0,2.5,3.0,3.5,German Pilsener,3.0
4,64883,Cauldron DIPA,7.7,johnmichaelsen,4.5,4.0,4.5,4.0,American Double / Imperial IPA,4.0


In [13]:
beer_data.filter(beer_data["beer_beerid"] == "14218").show()

+-----------+--------+------------------+------------+-------------+------------+-----------------+----------+--------------+
|beer_beerid|beer_abv|review_profilename|review_taste|review_palate|review_aroma|review_appearance|beer_style|review_overall|
+-----------+--------+------------------+------------+-------------+------------+-----------------+----------+--------------+
|      14218|     5.4|           branko7|         5.0|          4.0|         4.0|              4.0|    Kölsch|           5.0|
|      14218|     5.4|      TurdFurgison|         4.0|          3.5|         4.0|              4.0|    Kölsch|           4.5|
|      14218|     5.4|     BradLikesBrew|         3.5|          3.5|         4.0|              3.5|    Kölsch|           4.5|
|      14218|     5.4|          tai4ji2x|         4.0|          3.5|         4.0|              4.5|    Kölsch|           4.0|
|      14218|     5.4|       Halcyondays|         4.0|          4.0|         3.5|              3.0|    Kölsch|        

In [11]:
# StringIndexer ile kategorik sütunları sayılara dönüştürün
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(beer_data) for column in [ 'review_profilename', 'beer_style']]
pipeline = Pipeline(stages=indexers)
beer_data = pipeline.fit(beer_data).transform(beer_data)


In [12]:
beer_data.toPandas().head()

,beer_beerid,beer_name,beer_abv,review_profilename,review_taste,review_palate,review_aroma,review_appearance,beer_style,review_overall,review_profilename_index,beer_style_index
0,47986,Sausa Weizen,5.0,stcules,1.5,1.5,2.0,2.5,Hefeweizen,1.5,85.0,16.0
1,48213,Red Moon,6.2,stcules,3.0,3.0,2.5,3.0,English Strong Ale,3.0,85.0,72.0
2,48215,Black Horse Black Beer,6.5,stcules,3.0,3.0,2.5,3.0,Foreign / Export Stout,3.0,85.0,69.0
3,47969,Sausa Pils,5.0,stcules,3.0,2.5,3.0,3.5,German Pilsener,3.0,85.0,23.0
4,64883,Cauldron DIPA,7.7,johnmichaelsen,4.5,4.0,4.5,4.0,American Double / Imperial IPA,4.0,45.0,1.0


In [13]:
beer_data.toPandas().describe()

,beer_beerid,beer_abv,review_taste,review_palate,review_aroma,review_appearance,review_overall,review_profilename_index,beer_style_index
count,1.518478e+06,1.518478e+06,1.518478e+06,1.518478e+06,1.518478e+06,1.518478e+06,1.518478e+06,1.518478e+06,1.518478e+06
mean,2.140575e+04,7.042488e+00,3.804082e+00,3.753735e+00,3.746218e+00,3.850383e+00,3.823938e+00,2.205212e+03,2.391292e+01
std,2.178033e+04,2.322568e+00,7.286079e-01,6.793350e-01,6.953440e-01,6.143106e-01,7.172663e-01,3.777043e+03,2.280748e+01
min,5.000000e+00,1.000000e-02,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.654000e+03,5.200000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,2.450000e+02,5.000000e+00
50%,1.283000e+04,6.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,8.620000e+02,1.700000e+01
75%,3.923600e+04,8.500000e+00,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,2.445000e+03,3.700000e+01
max,7.731600e+04,5.770000e+01,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,3.290700e+04,1.030000e+02


In [14]:
from pyspark.ml.evaluation import RegressionEvaluator

# Veri kümesini eğitin
(training, test) = beer_data.randomSplit([0.8, 0.2])

# ALS modelini oluşturun
als = ALS(maxIter=10, rank=10,userCol='review_profilename_index', itemCol='beer_beerid',coldStartStrategy="drop", ratingCol='review_overall')

model = als.fit(training)
predictions = model.transform(test)
# Tüm kullanıcılar için önerileri alın
userRecs = model.recommendForAllUsers(2)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="review_overall", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}")



RMSE: 0.6158358784994143


In [ ]:
beer_data.filter(beer_data.review_profilename_index == 1).orderBy("review_overall").toPandas().tail(10)

In [18]:
# Önerileri gösterin
userRecs.toPandas().head(10)

,review_profilename_index,recommendations
0,1,"[(42588, 5.4334306716918945), (49042, 5.388335..."
1,12,"[(42588, 5.043178081512451), (49042, 4.9837236..."
2,13,"[(35361, 4.7431535720825195), (42588, 4.742771..."
3,22,"[(42588, 5.058685302734375), (49042, 5.0580124..."
4,26,"[(49042, 5.068743705749512), (42941, 5.0687437..."
5,27,"[(35361, 4.951788902282715), (13222, 4.9457111..."
6,28,"[(62900, 4.897942066192627), (35361, 4.8510613..."
7,31,"[(42588, 5.3660759925842285), (49042, 5.213920..."
8,34,"[(42588, 5.252553939819336), (49042, 5.2036590..."
9,44,"[(76768, 5.06256103515625), (42588, 4.99934530..."


In [16]:
userRecs = model.recommendForAllUsers(5)


In [17]:
userRecs.toPandas().to_csv("C:\\Users\\MertCan\\Desktop\\RECOMM\\userRecsFor5.csv", index=False)

In [20]:
pip install pandas pyodbc


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in c:\users\mertcan\anaconda\lib\site-packages (1.5.3)



In [22]:
import pyodbc

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-EFJ3DGUT\SQLEXPRESS01;'
                      'Database=BEER_RECOMMENDATION;'
)


In [23]:
conn

In [24]:
df = userRecs.toPandas()


In [27]:
cursor = conn.cursor()

for index, row in df.iterrows():
    recommendations_str = ', '.join([f"({item[0]}, {item[1]})" for item in row['recommendations']])
    cursor.execute("INSERT INTO recommendations (review_profilename_index, recommendations) VALUES (?, ?)",
                   row['review_profilename_index'], recommendations_str)

conn.commit()


In [28]:
beer_data_df = beer_data.toPandas()

In [31]:
beer_data_df.to_csv("C:\\Users\\MertCan\\Desktop\\RECOMM\\indexli.csv", index=False)